---
title: Encapsulation
abstract: |
    Compared to closures, C++ classes offer a more structured and flexible approach to encapsulating properties and methods into modular, reusable components. Constructors enable object initialization with varying properties, while access specifiers and member functions provide fine-grained control. Shared behavior and data can be managed through static members, and interactions between classes can be facilitated using non-member functions.
skip_execution: true
---

In [ ]:
from __init__ import *
!mkdir -p private

In [ ]:
if not input('Load JupyterAI? [Y/n]').lower()=='n':
    %reload_ext jupyter_ai

## Motivation

Recall an earlier function that increments a static variable `count`:

In [ ]:
%%cpp
unsigned int increment_count() {
    static int count;
    return ++count;
}

generate_n(ostream_iterator<int>(cout, " "), 5, increment_count);
generate_n(ostream_iterator<int>(cout, " "), 5, increment_count);

The above code uses [`std::generate_n`](https://en.cppreference.com/w/cpp/algorithm/generate_n.html) from `<algorithm>` and [`ostream_iterator`](https://en.cppreference.com/w/cpp/iterator/ostream_iterator/ostream_iterator.html) from `<iterator>` to print the 5 consecutive counts separated by spaces.

In [ ]:
%%ai
Explain in a paragraph or two what does the following code do:
---
generate_n(ostream_iterator<int>(cout, " "), 5, increment_count);

**How to make the funciton thread-safe?**

Closure can be used in place of the static variable, making the function thread-safe:

In [ ]:
%%cpp
std::function<int()> counter(int count=0) {
    return [count]() mutable {  // thread-safe as count is captured by copy
        return ++count;
    };
}

auto counter1=counter();
generate_n(ostream_iterator<int>(cout, " "), 5, counter1); cout << '\n';
generate_n(ostream_iterator<int>(cout, " "), 5, counter1); cout << '\n';

::::{exercise}
:label: ex:thread-safe

Why repeatedly calling 
```cpp
generate_n(ostream_iterator<int>(cout, " "), 5, counter1);
```
gives the same result but repeatedly calling 
```cpp
generate_n(ostream_iterator<int>(cout, " "), 5, increment_count);
```
does not?

::::

YOUR ANSWER HERE

**How to modify the count of an existing counter?**

One way is to modify the returned lambda expression to take an argument `new_count`:

In [ ]:
%%cpp
/**
 * @brief Creates a counter function that maintains internal state using a closure.
 * 
 * The returned lambda increments the counter by default, but allows resetting
 * the counter by passing a non-negative value.
 * 
 * @param count Initial value of the counter (default is 0).
 * @return std::function<int(int)> A function that takes an integer argument:
 *         - If new_count >= 0: ___
 *         - If new_count < 0: ___
 */
std::function<int(int)> counter(int count=0) {
    return [count](int new_count) mutable {
        return new_count>=0? (swap(count, new_count), new_count): ++count;
    };
}

auto counter1=counter();
for (auto i=0; i<5; i++) cout << counter1(-1) << ' '; cout << '\n';
counter1(10);
for (auto i=0; i<5; i++) cout << counter1(-1) << ' '; cout << '\n';
counter1(5);
for (auto i=0; i<5; i++) cout << counter1(-1) << ' '; cout << '\n';

::::{exercise}
:label: ex:new_count

Complete the above documentation of `counter` by filling in the blanks `___`.

::::

::::{solution} ex:new_count
:class: dropdown

```cpp
/**
 * @brief Creates a counter function that maintains internal state using a closure.
 * 
 * The returned lambda increments the counter by default, but allows resetting
 * the counter by passing a non-negative value.
 * 
 * @param count Initial value of the counter (default is 0).
 * @return std::function<int(int)> A function that takes an integer argument:
 *         - If new_count >= 0: resets the counter to new_count and return the previous count.
 *         - If new_count < 0: increments the counter and return the current count.
 */
std::function<int(int)> counter(int count = 0) {
    return [count](int new_count) mutable {
eturn new_count >= 0 ? (std::swap(count, new_count), new_count) : ++count;
    };
}
```
::::

::::{caution} Can we call `counter1()` to increment the count?

The call `counter1(-1)` to increment the count is awkward, as it relies on sentinel values, namely, negative counts. Unfortunately, C++ lambdas don’t support default arguments, so enabling `counter1()` without parameters isn’t straightforward. C++ being statically typed also makes the design harder to extend with additional features that may require different parameters and return values of different types.

::::

To resolve the issue, we will reimplement `counter` as a data type. `counter`is essentially a constructor that creates different counter objects `counter1` and `counter2`, each with their own `count` property.

## Class Definition

The following defines `Counter` as a data type, also known as a *class* or a *structure*:

```cpp
struct Counter {        // declaration list of members of the class Counter
    int count=0;        // a property that keeps an integer count, value-initialized to 0
    auto operator()() { // a method that defines the function call operator
        return ++count;
    }
}; // semi-colon required, unlike a compound statement
```

In [ ]:
%%cpp
struct Counter {
    int count=0;
    auto operator()() { return ++count; }
};

Counter counter1;
for (auto i=0; i<5; i++) cout << counter1() << ' '; cout << '\n';
counter1.count = 10;
for (auto i=0; i<5; i++) cout << counter1() << ' '; cout << '\n';
counter1.count = 5;
for (auto i=0; i<5; i++) cout << counter1() << ' '; cout << '\n';

The code declares a `counter1` object of type `Counter` and sets its count using `counter1.count = 10`. The custom data type `Counter` is defined with the [`struct`](https://en.cppreference.com/w/c/language/struct.html) keyword. Its members include:
- `count`, a *property* or *field*, behaves like a variable captured by value in a closure—each instance can hold a different value.
- `operator()`, a *method* or *member function*, resembles a lambda captured by reference—its definition is shared across instances. Implementing the [function call operator](https://en.cppreference.com/w/cpp/language/operators.html#Function_call_operator) makes the object a [functor](https://en.cppreference.com/w/cpp/functional.html), i.e., a callable object.

We can also separate the definition from the declarations as follows:

In [ ]:
%%cpp
struct Counter {
    int count=0;
    auto operator()();
};

In [ ]:
%%cpp
auto Counter::operator()() { return ++count; }

Counter counter1;
for (auto i=0; i<5; i++) cout << counter1() << ' '; cout << '\n';

Note that members are scoped using the class name to avoid name conflicts. The declaration can be placed in a header file, while the definition can be placed in a source file.

The following is an example, which also declares static members shared by all instances:
- A constant static property such as `DEFAULT_START` can be initialized inline just like a non-static property.
- However, a non-constant static property such as `start` cannot be initialized inline.
- A static method such as `reset_start` can be defined inline or separately just like a non-static method. However, it cannot access non-static members such as `count`, unlike a static method.

In [ ]:
%%writefile private/counter.hpp
#pragma once
struct Counter {
    int count=start-1;
    int operator()();

    // Static members shared by all instances
    static const int DEFAULT_START=1; // const initializable in header
    static int start;                 // mutable NOT initializable in header
    static void reset_start();  // static method can also be defined in header
};

/* ERROR: gnu/bin/ld: multiple definition of `Counter::start';
int Counter::start=Counter::DEFAULT_START; 
*/

In [ ]:
%%writefile private/counter.cpp
#include "counter.hpp"

int Counter::start=Counter::DEFAULT_START;           // initialization of mutable static member
void Counter::reset_start() { start=DEFAULT_START; } // cannot access non-static members like count
int Counter::operator()() { return ++count; }        // return type cannot be auto, to match the declaration

In [ ]:
%%writefile private/static.cpp
#include "counter.hpp"
#include <iostream>

using std::cout;

int main() {
    Counter::start=0;            // All instances see start as 0
    Counter counter1, counter2;
    for (auto i=0; i<5; i++) cout << counter1() << ' '; cout << '\n';
    for (auto i=0; i<5; i++) cout << counter2() << ' '; cout << '\n';
    Counter::reset_start();      // cannot be called as counter1.reset_start()
    Counter counter3, counter4;
    for (auto i=0; i<5; i++) cout << counter3() << ' '; cout << '\n';
    for (auto i=0; i<5; i++) cout << counter4() << ' '; cout << '\n';    
    return 0;
}

In [ ]:
!cd private && g++ static.cpp counter.cpp -o static && ./static

::::{exercise}
:label: ex:static

Explain how the static members work in the above code.

::::

YOUR ANSWER HERE

In [ ]:
%%ai
Explain in a paragraph or two why static constant but not mutable property can
be initialized inside the class declaration, and whether mutable property be 
initialized inside the same header file containing the class declaration?

::::{caution}

Due to an issue with the ROOT kernel, initialization of non-constant static variable does not work unfortunately:

```cpp
int Counter::start=Counter::DEFAULT_START;
```

::::

**How to initialize the count?**

Is there anything wrong with he following code?

In [ ]:
%%cpp
struct Counter {
    int count;
    auto operator()() { return ++count; }
};

Counter counter1;
for (auto i=0; i<5; i++) cout << counter1() << ' '; cout << '\n';

Unlike static/global variables, `int count;` may not [zero-initialize](https://en.cppreference.com/w/cpp/language/zero_initialization.html) the member `count`:

In [ ]:
%%writefile private/uninitialized.cpp
#include <iostream>

using std::cout;

struct Counter {
    int count;
    auto operator()() { return ++count; }
};

int main() {
    Counter counter1;
    cout << counter1() << '\n';
}

In [ ]:
!cd private && g++ -o uninitialized uninitialized.cpp && ./uninitialized

In [ ]:
%%ai
Explain in a table the rules for zero initialization in C++.

::::{exercise}
:label: ex:zero-init

Does the following code zero-initialize `count`?

```cpp
struct Counter {
    int count {};
    auto operator()() { return ++count; }
};
```

::::

YOUR ANSWER HERE

## Constructor

To initialize `count` to a possibly non-zero value, like the call to `auto counter2=counter(10);`, we need to define a constructor:

In [ ]:
%%cpp
struct Counter {
    int count;
    Counter(int count) {
        this->count = count;
    }
    auto operator()() { return ++count; }
};

generate_n(ostream_iterator<int>(cout, " "), 5, Counter(0));
Counter counter2(10);
for (auto i=0; i<5; i++) cout << counter2() << ' '; cout << '\n';

`Counter(int count)` defines a [constructor](https://en.cppreference.com/w/cpp/language/initializer_list.html) that takes an `int` argument and assigns it to the member `count`. Unlike ordinary functions, constructors omit a return type and a `return` statement—the constructed object is returned implicitly.

::::{caution}

Since the parameter `count` shadows the member `count`, `this->count` is used to explicitly reference the member `count` in `set_count`:

- [`this` pointer](https://en.cppreference.com/w/cpp/language/this.html) is used inside a method to points to the [implicit object parameter](https://en.cppreference.com/w/cpp/language/overload_resolution.html#Implicit_object_parameter), i.e., the current object.
- `this->count` is a [member access operator](https://en.cppreference.com/w/cpp/language/operator_member_access.html) that translates to `*this.count`.

::::

A simpler way to initialize `count` is to use a [member initializer list](https://en.cppreference.com/w/cpp/language/initializer_list.html#Member_initializer_list) `: count{count}` or `: count(count)` before the body of the constructor:

In [ ]:
%%cpp
struct Counter {
    int count;
    Counter(int count) : count{count} {}
    auto operator()() { return ++count; }
};

Counter counter2(10);  // direct initialization
for (auto i=0; i<5; i++) cout << counter2() << ' '; cout << '\n';
Counter counter3=20;   // copy initialization
for (auto i=0; i<5; i++) cout << counter3() << ' '; cout << '\n';

::::{tip}

The compiler knows that the first `count` in `: count{count}` must correspond to a property to initialize, so there is no need to use the `this` pointer.

::::

The copy initialization `Counter counter3=20;` may appear confusing as it converts an `int` to a `Counter` implicitly using the constructor `Counter(int count)`.

**How to make a constructor explicit?**

We can simply mark the constructor as [`explicit`](https://en.cppreference.com/w/cpp/language/explicit.html):

In [ ]:
%%cpp
struct Counter {
    int count;
    explicit Counter(int count) : count{count} {}
    auto operator()() { return ++count; }
};

Counter counter2(10);  // direct initialization
for (auto i=0; i<5; i++) cout << counter2() << ' '; cout << '\n';
// Counter counter3=20;   // error: no viable conversion from 'int' to 'Counter'

::::{exercise}
:label: ex:default_constructor

Why does the default initialization fails after declaring the constructor `Counter(int count)`?

```cpp
Counter counter1; // default initialization fails
```

:::{hint}

See [implicitly-declared default constructor](https://en.cppreference.com/w/cpp/language/default_constructor.html#Implicitly-declared_default_constructor).

:::

::::

YOUR ANSWER HERE

::::{exercise}
:label: ex:default_count

Modify the following code to force the compiler to create the [default constructor](https://en.cppreference.com/w/cpp/language/default_constructor.html) using the `default` keyword in the construction declaration or definition:
```cpp
class-name() = default;
class-name::class-name() = default;
```

::::

In [ ]:
%%cpp
struct Counter {
    int count=0;
    /*
    # REPLACE THE ENTIRE COMMENT WITH YOUR CODE #
    */
    Counter(int count) : count{count} {}
    auto operator()() { return ++count; }
};

Counter counter1, counter2(10);
for (auto i=0; i<5; i++) cout << counter1() << ' '; cout << '\n';
for (auto i=0; i<5; i++) cout << counter2() << ' '; cout << '\n';
for (auto i=0; i<5; i++) cout << counter1() << ' '; cout << '\n';

A even simpler fix is to make the parameter `count` in the constructor an optional parameter with a default value, which effectively defines the default constructor:

In [ ]:
%%cpp
struct Counter {
    int count;
    explicit Counter(int count=0) : count{count} {}
    auto operator()() { return ++count; }
};

Counter counter1, counter2(10);
for (auto i=0; i<5; i++) cout << counter1() << ' '; cout << '\n';
for (auto i=0; i<5; i++) cout << counter2() << ' '; cout << '\n';
for (auto i=0; i<5; i++) cout << counter1() << ' '; cout << '\n';

`Counter` now play the same role as the function `counter` defined at the beginning.

The compiler also implicitly declare the following if they are not user-defined:
- [destructor](https://en.cppreference.com/w/cpp/language/destructor.html#Implicitly-declared_destructor)
- [copy constructor](https://en.cppreference.com/w/cpp/language/copy_constructor.html#Implicitly-declared_copy_constructor)
- [copy assignment](https://en.cppreference.com/w/cpp/language/copy_assignment.html#Implicitly-declared_copy_assignment)
- [move constructor](https://en.cppreference.com/w/cpp/language/move_constructor.html#Implicitly-declared_move_constructor)
- [move assignment](https://en.cppreference.com/w/cpp/language/move_assignment.html#Implicitly-declared_move_assignment)

In [ ]:
%%cpp
Counter counter1;
for (auto i=0; i<5; i++) cout << counter1() << ' '; cout << '\n';
Counter counter2(counter1);  // implicitly declared copy constructor
for (auto i=0; i<5; i++) cout << counter2() << ' '; cout << '\n';
auto counter3=Counter(10);   // implicitly declared move constructor
for (auto i=0; i<5; i++) cout << counter3() << ' '; cout << '\n';
counter2=counter3;           // implicitly declared copy assignment
for (auto i=0; i<5; i++) cout << counter2() << ' '; cout << '\n';
counter1=Counter(10);        // implicitly declared move assignment
for (auto i=0; i<5; i++) cout << counter1() << ' '; cout << '\n';
counter1.~Counter();         // implicitly declared destructor
for (auto i=0; i<5; i++) cout << counter1() << ' '; cout << '\n';

## Member Access Specification

It is confusing that the `count` can go negative:

In [ ]:
%%cpp
struct Counter {
    int count;
    explicit Counter(int count=0) : count{count} {}
    auto operator()() { return ++count; }
};

Counter counter1(-10);
for (auto i=0; i<5; i++) cout << counter1() << ' '; cout << '\n';

**How to prevent users from setting the count to a negative number?**

One ways is to define a method `set_count` to validate the new count before setting it:

In [ ]:
%%cpp
struct Counter {
    int count;
    Counter(int count=0) : count{count} {}
    auto operator()() { return ++count; }
    void set_count(int count) {
        if (count>=0)
            this->count = count;
        else cerr << "The count must be non-negative.\n";
    }
};

Counter counter1;
counter1.set_count(-10);
for (auto i=0; i<5; i++) cout << counter1() << ' '; cout << '\n';

The method `void set_count(int count)` assigns the parameter to the member `count` only if it is non‑negative.

::::{exercise}
:label: ex:set_count

How to set the count to a negative number despite having the method `set_count`?

::::

YOUR ANSWER HERE

One ways is to define a method `set_count` to validate the new count before setting it:

```cpp
struct Counter {
public: // optional as public is the default label for struct
    Counter() = default;
    explicit Counter(int count) { set_count(count); } // use `set_count` to initialize `count`
    auto operator()() { return ++count; }
    void set_count(int count) {
        if (count>=0)
            this->count = count;
        else cerr << "The count must be non-negative.\n";
    }
private: // restrict direct access to members and friends
    int count=0; // a common default value for default/invalid initialization
};
```

In [ ]:
%%cpp
struct Counter {
    Counter() = default;
    explicit Counter(int count) { set_count(count); }
    auto operator()() { return ++count; }
    void set_count(int count) {
        if (count>=0)
            this->count = count;
        else cerr << "The count must be non-negative.\n";
    }
private:
    int count=0;
};

Counter counter1(-10);
counter1.set_count(-10);
// counter1.count=-10; // error: 'count' is a private member of 'Counter'
for (auto i=0; i<5; i++) cout << counter1() << ' '; cout << '\n';

- By listing `count` after the [private access specifier](https://en.cppreference.com/w/cpp/language/access.html#Private_member_access)
  ```cpp
  private : member-declarations
  ```
  `count` can be accessed only by class members or friends.
- `explicit Counter(int count) { set_count(count); }` initialize the `count` only if it is valid. We cannot use member initializer list `: count{set_count(count)}` since `set_count` does not return an `int`. If the parameter `count` is invalid, the member `count` remains `0` due to the initialization in the member declaration`int count=0;`.
- The default constructor is declared as `Counter() = default;` instead of `Counter(int count=0)` to reuse the same default value specified in the member declaration.

C++ introduced `class` which is similar to `struct` but uses the private access specifier as the default:

```cpp
class Counter {
private: // optional as private is the default label for class
    int count=0;
public:  // allow access anywhere
    Counter() = default;
    explicit Counter(int count) { set_count(count); }
    auto operator()() { return ++count; }
    void set_count(int count) {
        if (count>=0)
            this->count = count;
        else cerr << "The count must be non-negative.\n";
    }
};
```

In [ ]:
%%cpp
class Counter {
    int count=0;
public:
    Counter() = default;
    explicit Counter(int count) { set_count(count); }
    auto operator()() { return ++count; }
    void set_count(int count) {
        if (count>=0)
            this->count = count;
        else cerr << "The count must be non-negative.\n";
    }
};

Counter counter1(-10);
counter1.set_count(-10);
// counter1.count=-10; // error: 'count' is a private member of 'Counter'
for (auto i=0; i<5; i++) cout << counter1() << ' '; cout << '\n';

The members list after the [public access specifier](https://en.cppreference.com/w/cpp/language/access.html#Public_member_access) can be accessed anywhere.

Consider overloading the [increment operators](https://en.cppreference.com/w/cpp/language/operator_incdec.html) as follows:

In [ ]:
%%cpp
class Counter {
    int count=0;
public:
    Counter() = default;
    explicit Counter(int count) { set_count(count); }
    auto operator()() { return ++count; }
    void set_count(int count) {
        if (count>=0)
            this->count = count;
        else cerr << "The count must be non-negative.\n";
    }
    int operator++(int) { return count++; }
    int& operator++() { return ++count; }
};

Counter counter1;
for (auto i=0; i<5; i++) cout << ++counter1 << ' '; cout << '\n';
for (auto i=0; i<5; i++) cout << counter1++ << ' '; cout << '\n';
for (auto i=0; i<5; i++) cout << counter1() << ' '; cout << '\n';

- The `int` parameter in the postfix version `int operator++(int)` is dummy used to differentiate between prefix and postfix versions of the operators.[^dummy_int]
- Following in the pattern for the built-in operators, the prefix version `int& operator++()` return a reference and the postfix version `int operator++(int)` return a value. Instead of returning the `Counter`, it is more convenient to return the `count`, which can be easily printed and manipulated.

[^dummy_int]: The parameter is `0` by default, but can be changed by calling the operator explicitly as `counter1.operator++(2)`.

::::{exercise} 
:label: ex:prefix_increment

The code fails to ensure the `count` is non-negative. Why?

:::{hint}
:class: dropdown

```cpp
Counter counter1;
++counter1=-10;
for (auto i=0; i<5; i++) cout << counter1() << ' '; cout << '\n';
```

:::

::::

YOUR ANSWER HERE

A solution is as follows:

```cpp
class Counter {
    ...
public:
    ...
    Counter& operator++() { ++count; return *this; }
    Counter& operator=(int count) { set_count(count); return *this; }
};
```

- The prefix increment returns `Counter&` instead of `int&` to enforce the member access specification defined in `Counter`.
- The assignment operator is overloaded to reuse `set_count` to set the count.

In [ ]:
%%cpp
class Counter {
    int count=0;
public:
    Counter() = default;
    explicit Counter(int count) { set_count(count); }
    auto operator()() { return ++count; }
    void set_count(int count) {
        if (count>=0)
            this->count = count;
        else cerr << "The count must be non-negative.\n";
    }
    Counter& operator++() { ++count; return *this; }
    int operator++(int) { return count++; }
    Counter& operator=(int count) { set_count(count); return *this; }
};

Counter counter1(10);
++counter1=-10;
for (auto i=0; i<5; i++) cout << counter1() << ' '; cout << '\n';

::::{exercise}
:label: ex:decrement

Implements the prefix and postfix decrement operators for `Counter` properly to avoid negative `count`.

::::

In [ ]:
%%cpp
class Counter {
    int count=0;
public:
    Counter() = default;
    Counter(int count) { set_count(count); }
    auto operator()() { return ++count; }
    void set_count(int count) {
        if (count>=0)
            this->count = count;
        else cerr << "The count must be non-negative.\n";
    }
    Counter& operator++() { ++count; return *this; }
    int operator++(int) { return count++; }
    /*
    # REPLACE THE ENTIRE COMMENT WITH YOUR CODE #
    */
    Counter& operator=(int count) { set_count(count); return *this; }
};

Counter counter1;
counter1--;     // invalid (count==0)
--counter1=1;   // decrement is invalid but assignment is valid (count==1)
for (auto i=0; i<5; i++) cout << counter1() << ' '; cout << '\n';
counter1--;     // valid (count==5)
--counter1=-10; // decrement is valid (count==4) but assignment is invalid
for (auto i=0; i<5; i++) cout << counter1() << ' '; cout << '\n';

## Non-Member Method

Unfortunately, the following does not work since the operator `<<` does not handle `Counter` type:

```cpp
Counter counter1;
for (auto i=0; i<5; i++) cout << ++counter1 << ' '; cout << '\n';
for (auto i=0; i<5; i++) cout << counter1++ << ' '; cout << '\n';
```

One solution is to overload the `<<` operator:

```cpp
ostream& operator<<(ostream& os, const Counter &c) {
    return os << c.count;  // error: 'count' is a private member of 'Counter'
}
```

but this will fail as `count` is a private member of `Counter`, unless we declare it as a [friend](https://en.cppreference.com/w/cpp/language/friend.html) to `Counter`:

```cpp
class Counter {
    ...
public:
    ...
    friend ostream& operator<<(ostream& os, const Counter &c) {
        return os << c.count;
    }
};
```

In [ ]:
%%cpp
class Counter {
    int count=0;
public:
    Counter() = default;
    Counter(int count) { set_count(count); }
    auto operator()() { return ++count; }
    void set_count(int count) {
        if (count>=0)
            this->count = count;
        else cerr << "The count must be non-negative.\n";
    }
    Counter& operator++() { ++count; return *this; }
    int operator++(int) { return count++; }
    Counter& operator=(int count) { set_count(count); return *this; }
    friend ostream& operator<<(ostream& os, const Counter &c) {
        return os << c.count;
    }
};

Counter counter1;
for (auto i=0; i<5; i++) cout << ++counter1 << ' '; cout << '\n';
for (auto i=0; i<5; i++) cout << counter1++ << ' '; cout << '\n';
for (auto i=0; i<5; i++) cout << counter1() << ' '; cout << '\n';

Note that `operator<<` is a non-member function of `Counter`. To separate the friend declaration from its definition:

In [ ]:
%%cpp
class Counter {
    int count=0;
public:
    Counter() = default;
    Counter(int count) { set_count(count); }
    auto operator()() { return ++count; }
    void set_count(int count) {
        if (count>=0)
            this->count = count;
        else cerr << "The count must be non-negative.\n";
    }
    Counter& operator++() { ++count; return *this; }
    int operator++(int) { return count++; }
    Counter& operator=(int count) { set_count(count); return *this; }
    friend ostream& operator<<(ostream& os, const Counter &c);
};

In [ ]:
%%cpp
ostream& operator<<(ostream& os, const Counter &c) {
    return os << c.count;
}

Counter counter1;
for (auto i=0; i<5; i++) cout << ++counter1 << ' '; cout << '\n';
for (auto i=0; i<5; i++) cout << counter1++ << ' '; cout << '\n';
for (auto i=0; i<5; i++) cout << counter1() << ' '; cout << '\n';

In particular, the function signature for the definition is
```cpp
ostream& operator<<(ostream& os, const Counter &c) { ... }
```
but neither
- ```cpp
  ostream& Counter::operator<<(ostream& os, const Counter &c) { ... }
  ```
  since it is not a member of `Counter`; nor
- ```cpp
  friend ostream& operator<<(ostream& os, const Counter &c) { ... }
  ```
  since the friends of a class must be declared by the class.

::::{caution} Why not define `operator<<` as a member function of `Counter`?
:class: dropdown

We cannot because C++ resolves operators based on the left-hand type, which is `std::ostream` instead of `Counter` for `std::cout << counter1`.

::::

A simpler alternative is define to a [conversion operator](https://en.cppreference.com/w/cpp/language/cast_operator.html) `int()`:

```cpp
class Counter {
    ...
public:
    ...
    operator int() const { return count; }
};
```

Without marking the converter `int()` as `explicit`, it is considered in `cout << counter1++;` to implicitly convert `counter1` of type `Counter` to `int`, which can be handled by `<<` without implementing a new overload for `Counter`.

In [ ]:
%%cpp
class Counter {
    int count=0;
public:
    Counter() = default;
    Counter(int count) { set_count(count); }
    auto operator()() { return ++count; }
    void set_count(int count) {
        if (count>=0)
            this->count = count;
        else cerr << "The count must be non-negative.\n";
    }
    Counter& operator++() { ++count; return *this; }
    int operator++(int) { return count++; }
    Counter& operator=(int count) { set_count(count); return *this; }
    operator int() const { return count; }
};

Counter counter1;
for (auto i=0; i<5; i++) cout << ++counter1 << ' '; cout << '\n';
for (auto i=0; i<5; i++) cout << counter1++ << ' '; cout << '\n';
for (auto i=0; i<5; i++) cout << counter1() << ' '; cout << '\n';

::::{exercise}
:label: ex:sum_of_Counter

Implement addition of a `Counter` with another `Counter` or an `int`,
returning a new `Counter` whose count is the sum of the integer values of the operands.

::::

In [ ]:
%%cpp
class Counter {
    int count=0;
public:
    Counter() = default;
    explicit Counter(int count) { set_count(count); }
    auto operator()() { return ++count; }
    void set_count(int count) {
        if (count>=0)
            this->count = count;
        else cerr << "The count must be non-negative.\n";
    }
    Counter& operator++() { ++count; return *this; }
    int operator++(int) { return count++; }
    Counter& operator=(int count) { set_count(count); return *this; }
    operator int() const { return count; }
    /*
    # REPLACE THE ENTIRE COMMENT WITH YOUR CODE #
    */
};

Counter counter1, counter2(10), counter3=1+counter1+counter2+2;
for (auto i=0; i<5; i++) cout << counter3() << ' '; cout << '\n';
// 14 15 16 17 18